In [ ]:
#------------------------------------------------------------------------------------
# filename: 2_tile_image.ipynb.ipynb
# description: tile orthophotos of the Canton of Zurich into largest possible tiles
# Author: ChatGPT, Jan 30, 2023
# last updated: 12.2.23
#------------------------------------------------------------------------------------
#------------------------------------------------------#
#  THIS SECTION WAS CREATED BY CHATGPT (OpenAi, 2023) #
#----------------------------------------------------#
import os
import shlex
import subprocess
from osgeo import gdal

class TifProcessor:
    def __init__(self, input_folder_path, output_folder_path, tile_size):
        self.input_folder_path = input_folder_path
        self.output_folder_path = output_folder_path
        self.tile_size = tile_size

    def open_tif(self, file_path):
        return gdal.Open(file_path)

    def create_folder(self, folder_path):
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        print("new folder created: ", folder_path)
        return folder_path   
                
    def process_image(self, file_path, folder_path_out, filename):
        ds = self.open_tif(file_path)
        band = ds.GetRasterBand(1)
        xsize = band.XSize
        ysize = band.YSize

        
        new_folder_path = self.create_folder(os.path.join(folder_path_out, filename[:-4]))

        for i in range(0, xsize, self.tile_size):
            for j in range(0, ysize, self.tile_size):
                com_string = f"gdal_translate -of GTIFF -srcwin {i} {j} {self.tile_size} {self.tile_size} {file_path} {new_folder_path}/{filename[:-4]}_{i}_{j}.tif"

                # parallel tiling
                args = shlex.split(com_string)
                #print(args)
                p = subprocess.Popen(args)
                p.wait()
                
        print("image", filename, "successfully tiled.")
        ds = None
        band = None
        
folder_path_in = '../../_data/with_pv/'
folder_path_out = '../../_data/with_pv_625'
tile_size = 625 
errors = []

# create an instance of the class
tif_processor = TifProcessor(input_folder_path=folder_path_in, 
                             output_folder_path=folder_path_out, 
                             tile_size=tile_size)

for filename in os.listdir(folder_path_in):
    print(filename)
    if filename.endswith(".tif"):
        # check if folder exists
        folder_to_check = filename[:-4]
        folder_path = os.path.join(folder_path_out, folder_to_check)
        if os.path.exists(folder_path) and os.path.isdir(folder_path) and os.listdir(folder_path):
            print(f"Folder {folder_to_check} exists in {folder_path_out} and contains files.")
        else:
            try:
                # create tiled images from raw_images/rgb_images
                file_path = os.path.join(folder_path_in, filename)
                # call the process_image method on the class instance
                tif_processor.process_image(file_path=file_path, folder_path_out=folder_path_out, filename=filename)
                
            except Exception as e:
                errors.append((filename, e))
                print(f"Error in file {filename}: {e}")
                continue
